<div style="border-radius: 15px 0 15px 0px; border: 2px solid #f1c40f; padding: 10px; background-color: #191970; text-align: center; box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.2);">
    <h1 style="color: #f1c40f; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 24px;">Makine Öğrenmesiyle Öğrencilerin Yaşam Alışkanlıkları ve Akademik Durumlarına Göre Depresyon Riskinin Tahmini</h1>
</div>

# **1) Proje Tanımı**

        Amaç, öğrencilerin yaşam tarzı alışkanlıkları (uyku, sosyal aktivite, stres, fiziksel aktivite, çalışma saati vb.) ve akademik performans  göstergeleri (CGPA, ders çalışma saatleri, akademik tatmin) kullanılarak depresyon yaşayıp yaşamadıklarını tahmin eden bir makine öğrenmesi modeli geliştirmektir.

# **2) Veri Kaynakları**

## **2.1) Dataset 1** : Daily Lifestyle and Academic Performance

- 2.000 öğrenciden Google Form anketiyle toplanan verileri içerir.

- Ders çalışma süresi, sosyal etkinlikler, uyku, sosyalleşme, fiziksel aktivite, stres düzeyi ve not ortalaması bilgilerini kapsar.

- Veriler, Ağustos 2023 - Mayıs 2024 akademik yılına aittir.

- Katılımcıların çoğu Hindistan’daki öğrencilerden oluşmaktadır.

- **Amaç:** Günlük alışkanlıkların akademik başarı ve öğrenci refahı üzerindeki etkisini incelemek.

## 2.2) Dataset 2 : Student Mental Health

- Öğrencilerin ruh sağlığı ve depresyon riskini etkileyen faktörleri kapsamlı şekilde içerir.

- Demografik bilgiler, akademik ve iş baskıları, yaşam tarzı alışkanlıkları ve ruh sağlığı göstergelerini kapsar.

- **Amaç:** Depresyon eğilimlerini ve risk faktörlerini analiz etmek.

# **3) Yüklemeler ve İlk Kontroller**

### Paketlerin Yüklenmesi

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.ensemble import RandomForestRegressor

### Veri Yükleme

In [2]:
df1 = pd.read_csv("./datasets/student_lifestyle_dataset.csv")
df2 = pd.read_csv("./datasets/student_depression_dataset.csv")

### Temel Veri Bilgileri

In [3]:
print("\nDataset 1")
print(f"Shape (rows, columns): {df1.shape}")
print("\nFirst 5 rows:")
display(df1.head())

print("\n\n")

print("\nDataset 2")
print(f"Shape (rows, columns): {df2.shape}")
print("\nFirst 5 rows:")
display(df2.head())


Dataset 1
Shape (rows, columns): (2000, 8)

First 5 rows:


,Student_ID,Study_Hours_Per_Day,Extracurricular_Hours_Per_Day,Sleep_Hours_Per_Day,Social_Hours_Per_Day,Physical_Activity_Hours_Per_Day,GPA,Stress_Level
0,1,6.9,3.8,8.7,2.8,1.8,2.99,Moderate
1,2,5.3,3.5,8.0,4.2,3.0,2.75,Low
2,3,5.1,3.9,9.2,1.2,4.6,2.67,Low
3,4,6.5,2.1,7.2,1.7,6.5,2.88,Moderate
4,5,8.1,0.6,6.5,2.2,6.6,3.51,High






Dataset 2
Shape (rows, columns): (27901, 18)

First 5 rows:


,id,Gender,Age,City,Profession,Academic Pressure,Work Pressure,CGPA,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,2,Male,33.0,Visakhapatnam,Student,5.0,0.0,8.97,2.0,0.0,'5-6 hours',Healthy,B.Pharm,Yes,3.0,1.0,No,1
1,8,Female,24.0,Bangalore,Student,2.0,0.0,5.90,5.0,0.0,'5-6 hours',Moderate,BSc,No,3.0,2.0,Yes,0
2,26,Male,31.0,Srinagar,Student,3.0,0.0,7.03,5.0,0.0,'Less than 5 hours',Healthy,BA,No,9.0,1.0,Yes,0
3,30,Female,28.0,Varanasi,Student,3.0,0.0,5.59,2.0,0.0,'7-8 hours',Moderate,BCA,Yes,4.0,5.0,Yes,1
4,32,Female,25.0,Jaipur,Student,4.0,0.0,8.13,3.0,0.0,'5-6 hours',Moderate,M.Tech,Yes,1.0,1.0,No,0


### Genel Bilgi

In [4]:
print("\nDataset 1")
display(df1.info())

print("\n\n")

print("\nDataset 2")
display(df2.info())


Dataset 1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 8 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Student_ID                       2000 non-null   int64  
 1   Study_Hours_Per_Day              2000 non-null   float64
 2   Extracurricular_Hours_Per_Day    2000 non-null   float64
 3   Sleep_Hours_Per_Day              2000 non-null   float64
 4   Social_Hours_Per_Day             2000 non-null   float64
 5   Physical_Activity_Hours_Per_Day  2000 non-null   float64
 6   GPA                              2000 non-null   float64
 7   Stress_Level                     2000 non-null   object 
dtypes: float64(6), int64(1), object(1)
memory usage: 125.1+ KB


None





Dataset 2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27901 entries, 0 to 27900
Data columns (total 18 columns):
 #   Column                                 Non-Null Count  Dtype  
---  ------                                 --------------  -----  
 0   id                                     27901 non-null  int64  
 1   Gender                                 27901 non-null  object 
 2   Age                                    27901 non-null  float64
 3   City                                   27901 non-null  object 
 4   Profession                             27901 non-null  object 
 5   Academic Pressure                      27901 non-null  float64
 6   Work Pressure                          27901 non-null  float64
 7   CGPA                                   27901 non-null  float64
 8   Study Satisfaction                     27901 non-null  float64
 9   Job Satisfaction                       27901 non-null  float64
 10  Sleep Duration                         27901 non-null  o

None

### İstatiksel Özeti

#### Dataset 1

In [5]:
display(df1.drop(['Student_ID'],axis=1).describe().T)

,count,mean,std,min,25%,50%,75%,max
Study_Hours_Per_Day,2000.0,7.47580,1.423888,5.00,6.3,7.40,8.70,10.0
Extracurricular_Hours_Per_Day,2000.0,1.99010,1.155855,0.00,1.0,2.00,3.00,4.0
Sleep_Hours_Per_Day,2000.0,7.50125,1.460949,5.00,6.2,7.50,8.80,10.0
Social_Hours_Per_Day,2000.0,2.70455,1.688514,0.00,1.2,2.60,4.10,6.0
Physical_Activity_Hours_Per_Day,2000.0,4.32830,2.514110,0.00,2.4,4.10,6.10,13.0
GPA,2000.0,3.11596,0.298674,2.24,2.9,3.11,3.33,4.0


1. **Veri Kümesi Boyutları**:
   - Satir: 2,000
   - Sütun: 8
   - Sütunların hiçbirinde eksik değere rastlanmadı.

2. **Veri Türleri**:
   - Veri kümesi şunları içerir:
     - Sayısal sütunlar: 6 (örneğin, aktivitelere harcanan saatler ve not ortalaması).
     - Kategorik sütun: 1 (Stres Düzeyi).
     - ID Sütunu: 1 (Student_ID).

#### **İlk Gözlemler**
1. **Dengeli Yaşam Tarzı**:
   - Öğrencilerin ders çalışmaya, uykuya ve fiziksel aktiviteye harcadıkları ortalama saatlerin birbirine yakın olması, öğrenciler arasında nispeten dengeli bir yaşam tarzının olduğunu göstermektedir.

2. **GPA ve Yaşam Tarzı Korelasyonu**:
   - Nispeten yüksek ortalama not ortalaması, dengeli yaşam tarzı alışkanlıkları (örneğin yeterli uyku ve çalışma) ile akademik performans arasında olası pozitif bir ilişki olduğunu düşündürmektedir.

3. **Stres Seviyeleri**:
   - Kategorik bir özellik olarak, bunun GPA ve diğer değişkenlerle korelasyonları daha detaylı incelenebilir.

4. **Aykırı Değerler**:
   - Fiziksel aktivite saatleri (maksimum: 13 saat) bazı öğrencilerin olağanüstü derecede aktif olduğunu göstermektedir.
   - Sosyalleşme saatleri de değişkendir ve günlük 6 saate kadar çıkabilir, bu da çeşitli sosyal alışkanlıkların göstergesidir.

#### Dataset 2

In [6]:
display(df2.drop(['id','Depression'],axis=1).describe().T)

,count,mean,std,min,25%,50%,75%,max
Age,27901.0,25.822300,4.905687,18.0,21.00,25.00,30.00,59.0
Academic Pressure,27901.0,3.141214,1.381465,0.0,2.00,3.00,4.00,5.0
Work Pressure,27901.0,0.000430,0.043992,0.0,0.00,0.00,0.00,5.0
CGPA,27901.0,7.656104,1.470707,0.0,6.29,7.77,8.92,10.0
Study Satisfaction,27901.0,2.943837,1.361148,0.0,2.00,3.00,4.00,5.0
Job Satisfaction,27901.0,0.000681,0.044394,0.0,0.00,0.00,0.00,4.0
Work/Study Hours,27901.0,7.156984,3.707642,0.0,4.00,8.00,10.00,12.0


1. **Veri Kümesi Boyutları**
    - Satır: 27,901
    - Sütun: 9
    - Eksik Değer: Hiçbir sütunda eksik değer bulunmamaktadır.

2. **Veri Türleri**
   - Sayısal sütunlar: 9 (yaş, akademik baskı, iş baskısı, CGPA, çalışma memnuniyeti, iş memnuniyeti, çalışma/öğrenim saatleri vb.)
   - Kategorik sütunlar: 8 (cinsiyet,şehir,beslenme alışkanlıkları, family history vb.)
   - ID Sütunu: 1 (ID)

#### **İlk Gözlemler**

1. **Demografik Dağılım (Age)**
   - Ortalama yaş 25.82, minimum 18, maksimum 59 olarak gözlenmiştir.
   - Bu, veri kümesinde hem üniversite öğrencilerini hem de mezun/çalışan bireyleri içeren geniş bir yaş aralığına işaret etmektedir.

2. **Akademik Baskı ve CGPA**
   - Akademik Baskı (mean = 3.14) orta seviyededir.
   - CGPA ortalaması 7.65 olup, bu değer oldukça iyi bir akademik performansa işaret eder.
   - Bu iki değişkenin birlikte incelenmesi, akademik baskının başarı üzerindeki etkisini anlamada kritik olacaktır.

3. **Çalışma Memnuniyeti (Study Satisfaction)**
   - Ortalama 2.94, medyan 3.0, bu da memnuniyetin orta düzeyde olduğunu gösterir.
   - Akademik baskı ve çalışma memnuniyeti birlikte analiz edildiğinde, baskı arttıkça memnuniyetin azalıp azalmadığı değerlendirilebilir.

4. **Aykırı Değerler**
   - Work Pressure ve Job Satisfaction değişkenlerinde 5 ve 4 değerlerine kadar çıkan seyrek yüksek değerler, aykırı gözlemler olabilir.
   - Age değişkeninde 59 yaşa kadar çıkılması da, bazı katılımcıların öğrenci dışı profilde olabileceğini düşündürmektedir. 

#  **4) Birleştirme Stratejisi**

**Amaç:** Birleştirme sonucunda, öğrencilerin günlük yaşam alışkanlıklarının (uyku, çalışma süresi, sosyal aktivite vb.) depresyon riski üzerindeki etkisini analiz etmek.

## 4.1. Ortak ID Anahtar Üzerinden


   - ### Sütun İsimlerini Kontrol Etme

In [7]:
print("Dataset 1: ",df1.columns)

print("\n")

print("Dataset 2: ",df2.columns)

Dataset 1:  Index(['Student_ID', 'Study_Hours_Per_Day', 'Extracurricular_Hours_Per_Day',
       'Sleep_Hours_Per_Day', 'Social_Hours_Per_Day',
       'Physical_Activity_Hours_Per_Day', 'GPA', 'Stress_Level'],
      dtype='object')


Dataset 2:  Index(['id', 'Gender', 'Age', 'City', 'Profession', 'Academic Pressure',
       'Work Pressure', 'CGPA', 'Study Satisfaction', 'Job Satisfaction',
       'Sleep Duration', 'Dietary Habits', 'Degree',
       'Have you ever had suicidal thoughts ?', 'Work/Study Hours',
       'Financial Stress', 'Family History of Mental Illness', 'Depression'],
      dtype='object')


  #### Her iki veri setinde de bir benzersiz kimlik var. Bu, bir “anahtar” gibi kullanılacaktır.

- ### Sütun İsimlerini Düzenle
     Kolaylık için id sütununu Student_ID olarak yeniden adlandıralım

In [8]:
df2.rename(columns={'id': 'Student_ID'}, inplace=True)

- ### Ortak Anahtar Üzerinden Birleştirme
      df1’deki tüm öğrenciler, df2’de varsa mental sağlık bilgisiyle eşleşir.

In [9]:
merged_df = pd.merge(df1, df2, on='Student_ID', how='left')

In [10]:
print("\nBirleştirilmiş veri seti boyutu:", merged_df.shape)


Birleştirilmiş veri seti boyutu: (2000, 25)


- ### Nihai veri seti kaydetme

In [11]:
merged_df.to_csv("./datasets/Merged_Student_Lifestyle_MentalHealth.csv", index=False)

<div style="border-radius: 15px 0 15px 0px; border: 2px solid #f1c40f; padding: 10px; background-color: #90e0ef; text-align: center; box-shadow: 0px 2px 4px rgba(0, 0, 0, 0.2);">
    <h1 style="color: #38040e; text-shadow: 2px 2px 4px rgba(0, 0, 0, 0.5); font-weight: bold; margin-bottom: 10px; font-size: 24px;">Keşifsel Veri Analizi (EDA)</h1>
</div>

- ## **Amaç:** Verinin yapısını anlamak, türleri, eksik değerleri, çift kayıtları görmek vb.

### 1. İlk 5 satır

In [12]:
merged_df.head()

,Student_ID,Study_Hours_Per_Day,Extracurricular_Hours_Per_Day,Sleep_Hours_Per_Day,Social_Hours_Per_Day,Physical_Activity_Hours_Per_Day,GPA,Stress_Level,Gender,Age,...,Study Satisfaction,Job Satisfaction,Sleep Duration,Dietary Habits,Degree,Have you ever had suicidal thoughts ?,Work/Study Hours,Financial Stress,Family History of Mental Illness,Depression
0,1,6.9,3.8,8.7,2.8,1.8,2.99,Moderate,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,5.3,3.5,8.0,4.2,3.0,2.75,Low,Male,33.0,...,2.0,0.0,'5-6 hours',Healthy,B.Pharm,Yes,3.0,1.0,No,1.0
2,3,5.1,3.9,9.2,1.2,4.6,2.67,Low,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,6.5,2.1,7.2,1.7,6.5,2.88,Moderate,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,8.1,0.6,6.5,2.2,6.6,3.51,High,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### 1.2 Boyut ve sütun tipleri

In [13]:
print("Shape:", merged_df.shape)
print("\n",merged_df.dtypes)

Shape: (2000, 25)

 Student_ID                                 int64
Study_Hours_Per_Day                      float64
Extracurricular_Hours_Per_Day            float64
Sleep_Hours_Per_Day                      float64
Social_Hours_Per_Day                     float64
Physical_Activity_Hours_Per_Day          float64
GPA                                      float64
Stress_Level                              object
Gender                                    object
Age                                      float64
City                                      object
Profession                                object
Academic Pressure                        float64
Work Pressure                            float64
CGPA                                     float64
Study Satisfaction                       float64
Job Satisfaction                         float64
Sleep Duration                            object
Dietary Habits                            object
Degree                                    object


### 1.3 Özet istatistik (sayısal)


In [14]:
merged_df.drop(['Student_ID','Depression'],axis=1).describe().T

,count,mean,std,min,25%,50%,75%,max
Study_Hours_Per_Day,2000.0,7.475800,1.423888,5.00,6.300,7.40,8.700,10.00
Extracurricular_Hours_Per_Day,2000.0,1.990100,1.155855,0.00,1.000,2.00,3.000,4.00
Sleep_Hours_Per_Day,2000.0,7.501250,1.460949,5.00,6.200,7.50,8.800,10.00
Social_Hours_Per_Day,2000.0,2.704550,1.688514,0.00,1.200,2.60,4.100,6.00
Physical_Activity_Hours_Per_Day,2000.0,4.328300,2.514110,0.00,2.400,4.10,6.100,13.00
GPA,2000.0,3.115960,0.298674,2.24,2.900,3.11,3.330,4.00
Age,399.0,25.593985,4.945488,18.00,21.000,25.00,29.500,39.00
Academic Pressure,399.0,3.087719,1.403450,1.00,2.000,3.00,4.000,5.00
Work Pressure,399.0,0.000000,0.000000,0.00,0.000,0.00,0.000,0.00
CGPA,399.0,7.611604,1.462645,5.11,6.165,7.70,8.935,9.97


### 1.4 Duplicate kontrolü (aynı Student_ID)

In [15]:
merged_df[merged_df.duplicated(subset='Student_ID', keep=False)].shape

(0, 25)

### 1.5 Kategorik değişkenlerin frekansları

In [16]:
for col in merged_df.select_dtypes(include='object').columns:
    print(f"\n--- {col} ---")
    print(merged_df[col].value_counts(dropna=False).head(20))


--- Stress_Level ---
Stress_Level
High        1029
Moderate     674
Low          297
Name: count, dtype: int64

--- Gender ---
Gender
NaN       1601
Male       216
Female     183
Name: count, dtype: int64

--- City ---
City
NaN            1601
Lucknow          24
Kalyan           23
Srinagar         21
Thane            18
Kolkata          18
Vasai-Virar      18
Ludhiana         16
Agra             16
Hyderabad        15
Bhopal           15
Surat            15
Rajkot           15
Pune             14
Jaipur           14
Mumbai           14
Ahmedabad        14
Vadodara         13
Ghaziabad        12
Varanasi         12
Name: count, dtype: int64

--- Profession ---
Profession
NaN                 1601
Student              398
'Civil Engineer'       1
Name: count, dtype: int64

--- Sleep Duration ---
Sleep Duration
NaN                    1601
'Less than 5 hours'     120
'7-8 hours'             106
'5-6 hours'              92
'More than 8 hours'      81
Name: count, dtype: int64

--- Dietary

- ## Eksik veriler (Missing values)

### **Amaç:** Hangi sütunlarda kaç tane eksik var, eksiklerin yapısına göre strateji seçecektir.

### 2.1 Eksik değer sayısı ve oranı

In [17]:
miss = merged_df.isnull().sum().sort_values(ascending=False)
miss_percent = (miss / len(merged_df) * 100).round(2)
pd.concat([miss, miss_percent], axis=1, keys=['count', 'percent']).head(30)

,count,percent
Gender,1601,80.05
Job Satisfaction,1601,80.05
Study Satisfaction,1601,80.05
CGPA,1601,80.05
Work Pressure,1601,80.05
Academic Pressure,1601,80.05
Profession,1601,80.05
City,1601,80.05
Age,1601,80.05
Sleep Duration,1601,80.05


## Eksik oranı %50 den yüksek ve projeye katkısı olmayan tüm sütunlar çıkarılacaktır

### Çıkarılan sütunlar

In [18]:
merged_df = merged_df.drop(columns=['Gender','Job Satisfaction','CGPA','Profession','Sleep Duration','Age',
                                    'Have you ever had suicidal thoughts ?','Degree','City','Work/Study Hours','Dietary Habits',
                                   'Family History of Mental Illness','Financial Stress','Work Pressure','Study Satisfaction','Student_ID'])

In [19]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Study_Hours_Per_Day              2000 non-null   float64
 1   Extracurricular_Hours_Per_Day    2000 non-null   float64
 2   Sleep_Hours_Per_Day              2000 non-null   float64
 3   Social_Hours_Per_Day             2000 non-null   float64
 4   Physical_Activity_Hours_Per_Day  2000 non-null   float64
 5   GPA                              2000 non-null   float64
 6   Stress_Level                     2000 non-null   object 
 7   Academic Pressure                399 non-null    float64
 8   Depression                       399 non-null    float64
dtypes: float64(8), object(1)
memory usage: 140.8+ KB


### HEM 'Akademik Baskı' HEM de 'Depresyon'un NaN olduğu 995 satırları silme

In [20]:
# Her iki sütunun da NaN olduğu satırları belirleme
mask = merged_df['Academic Pressure'].isna() & merged_df['Depression'].isna()

# Bu satırların indekslerini alma
nan_indices = merged_df[mask].index

# Bunlardan 700 tanesini rastgele seçin (yeterli değilse daha azını)
drop_indices = nan_indices.to_series().sample(995, random_state=42).index

# Sadece 700 satırı bırak
merged_df = merged_df.drop(index=drop_indices)

# sonuç
print(merged_df.info())


<class 'pandas.core.frame.DataFrame'>
Index: 1005 entries, 1 to 1999
Data columns (total 9 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Study_Hours_Per_Day              1005 non-null   float64
 1   Extracurricular_Hours_Per_Day    1005 non-null   float64
 2   Sleep_Hours_Per_Day              1005 non-null   float64
 3   Social_Hours_Per_Day             1005 non-null   float64
 4   Physical_Activity_Hours_Per_Day  1005 non-null   float64
 5   GPA                              1005 non-null   float64
 6   Stress_Level                     1005 non-null   object 
 7   Academic Pressure                399 non-null    float64
 8   Depression                       399 non-null    float64
dtypes: float64(8), object(1)
memory usage: 78.5+ KB
None


In [21]:
miss = merged_df.isnull().sum().sort_values(ascending=False)
miss_percent = (miss / len(merged_df) * 100).round(2)
pd.concat([miss, miss_percent], axis=1, keys=['count', 'percent']).head(30)

,count,percent
Academic Pressure,606,60.3
Depression,606,60.3
Study_Hours_Per_Day,0,0.0
Extracurricular_Hours_Per_Day,0,0.0
Sleep_Hours_Per_Day,0,0.0
Physical_Activity_Hours_Per_Day,0,0.0
Social_Hours_Per_Day,0,0.0
Stress_Level,0,0.0
GPA,0,0.0


### Yeni dataframe'in boyutu

In [22]:
merged_df.shape

(1005, 9)

In [23]:
merged_df.sample(5)

,Study_Hours_Per_Day,Extracurricular_Hours_Per_Day,Sleep_Hours_Per_Day,Social_Hours_Per_Day,Physical_Activity_Hours_Per_Day,GPA,Stress_Level,Academic Pressure,Depression
1941,8.2,1.2,5.1,5.3,4.2,3.19,High,NaN,NaN
909,7.4,3.9,8.8,1.0,2.9,3.11,Moderate,1.0,1.0
1048,6.1,2.6,8.0,5.5,1.8,3.01,Moderate,3.0,1.0
1447,8.6,3.6,5.6,5.9,0.3,3.21,High,NaN,NaN
699,9.1,0.7,5.2,1.2,7.8,3.64,High,NaN,NaN


# Stres Düzeyi için Etiket Kodlaması (Label Encoding)

In [24]:
merged_df['Stress_Level'] = merged_df['Stress_Level'].map({
    'Low': 0,
    'Moderate': 1,
    'High': 2
})

In [25]:
merged_df.sample(6)

,Study_Hours_Per_Day,Extracurricular_Hours_Per_Day,Sleep_Hours_Per_Day,Social_Hours_Per_Day,Physical_Activity_Hours_Per_Day,GPA,Stress_Level,Academic Pressure,Depression
1768,8.1,3.3,7.0,1.2,4.4,3.80,2,1.0,0.0
1035,6.0,1.2,9.9,2.5,4.4,2.88,1,NaN,NaN
715,8.3,3.0,8.8,2.3,1.6,2.92,2,1.0,0.0
876,9.7,1.2,8.9,3.2,1.0,3.26,2,4.0,1.0
509,8.2,1.6,8.6,1.1,4.5,3.46,2,3.0,1.0
273,9.8,0.4,8.4,2.4,3.0,3.26,2,3.0,1.0


In [26]:
miss = merged_df.isnull().sum()
miss

Study_Hours_Per_Day                  0
Extracurricular_Hours_Per_Day        0
Sleep_Hours_Per_Day                  0
Social_Hours_Per_Day                 0
Physical_Activity_Hours_Per_Day      0
GPA                                  0
Stress_Level                         0
Academic Pressure                  606
Depression                         606
dtype: int64

# Akademik Baskı sütunundaki eksik değerler tahmin etme

  - # Strateji: Makine Öğrenmesi Tabanlı Tahmini Kullanması
       ### Diğer özelliklere dayanarak Akademik Baskıyı tahmin etmek için küçük bir regresyon/sınıflandırma modeli oluşturacağız.

#### Bilinen ve bilinmeyen Akademik Baskı ayırma

In [27]:
known_ap = merged_df[merged_df['Academic Pressure'].notnull()]
unknown_ap = merged_df[merged_df['Academic Pressure'].isnull()]

#### Akademik Baskıyı Tahmin Etmeye Yardımcı Özellikler

In [28]:
features = ['Study_Hours_Per_Day', 'Extracurricular_Hours_Per_Day',
            'Sleep_Hours_Per_Day', 'Social_Hours_Per_Day',
            'Physical_Activity_Hours_Per_Day', 'GPA', 'Stress_Level']

#### Bilinen veriler üzerinde model eğitimi

In [29]:
X_known = known_ap[features]
y_known = known_ap['Academic Pressure']

#### Küçük eksik boşlukları geçici olarak doldurma (ortalama/medyan)

In [30]:
imputer = SimpleImputer(strategy='median')
X_known = imputer.fit_transform(X_known)

In [31]:
model_ap = RandomForestRegressor(random_state=42)
model_ap.fit(X_known, y_known)

RandomForestRegressor(random_state=42)

#### Eksik değerleri tahmin etme

In [32]:
X_unknown = imputer.transform(unknown_ap[features])
predicted_ap = model_ap.predict(X_unknown)

#### Tahmin edilen değerleri atama (en yakın Likert ölçeğine yuvarlanmış)

In [33]:
merged_df.loc[merged_df['Academic Pressure'].isnull(), 'Academic Pressure'] = predicted_ap.round().clip(1, 5)

### Özet İstatistikler

In [34]:
merged_df.describe().T

,count,mean,std,min,25%,50%,75%,max
Study_Hours_Per_Day,1005.0,7.413035,1.446737,5.00,6.10,7.40,8.60,10.0
Extracurricular_Hours_Per_Day,1005.0,2.012836,1.150275,0.00,1.00,2.00,3.00,4.0
Sleep_Hours_Per_Day,1005.0,7.523980,1.464647,5.00,6.30,7.60,8.80,10.0
Social_Hours_Per_Day,1005.0,2.696119,1.677949,0.00,1.30,2.50,4.10,6.0
Physical_Activity_Hours_Per_Day,1005.0,4.354030,2.505249,0.00,2.40,4.10,6.20,12.4
GPA,1005.0,3.109632,0.304782,2.28,2.89,3.11,3.32,4.0
Stress_Level,1005.0,1.324378,0.748340,0.00,1.00,1.00,2.00,2.0
Academic Pressure,1005.0,3.101493,0.966391,1.00,3.00,3.00,4.00,5.0
Depression,399.0,0.566416,0.496191,0.00,0.00,1.00,1.00,1.0


## Bu şekilde:

   - #### Stres, not ortalaması ve aktivite seviyeleri arasındaki korelasyonları korumaktadır.

   - #### Sadece ortalama/medyan tahminini kullanarak oluşabilecek önyargılardan kaçınmaktadır.

   - #### Akademik Baskıyı gerçekçi (1-5 arasında) tutmaktadır.

In [35]:
# Use the 399 labeled rows to train a depression model
known_dep = merged_df[merged_df['Depression'].notnull()]
unknown_dep = merged_df[merged_df['Depression'].isnull()]

X_known = known_dep.drop(columns=['Depression'])
y_known = known_dep['Depression']

# Simple imputation
X_known = imputer.fit_transform(X_known)
X_unknown = imputer.transform(unknown_dep.drop(columns=['Depression']))

# Train model
from sklearn.ensemble import RandomForestClassifier
model_dep = RandomForestClassifier(random_state=42)
model_dep.fit(X_known, y_known)

# Predict depression for unlabeled data
predicted_dep = model_dep.predict(X_unknown)

# Store predictions in a new column
merged_df.loc[merged_df['Depression'].isnull(), 'Predicted_Depression'] = predicted_dep


In [37]:
print(merged_df['Academic Pressure'].value_counts(normalize=True))
print(merged_df['Depression'].value_counts(normalize=True))
print(merged_df['Predicted_Depression'].value_counts(normalize=True))

Academic Pressure
3.0    0.549254
4.0    0.175124
2.0    0.111443
5.0    0.091542
1.0    0.072637
Name: proportion, dtype: float64
Depression
1.0    0.566416
0.0    0.433584
Name: proportion, dtype: float64
Predicted_Depression
1.0    0.760726
0.0    0.239274
Name: proportion, dtype: float64


In [38]:
merged_df['Depression'] = merged_df['Depression'].fillna(merged_df['Predicted_Depression'])

In [39]:
# Re-check balance
merged_df['Depression'].value_counts(normalize=True)

Depression
1.0    0.683582
0.0    0.316418
Name: proportion, dtype: float64

In [40]:
# Current proportion
current_ratio = merged_df['Depression'].mean()  # Mean works because 1=depressed, 0=not
target_ratio = 0.566  # Original true ratio

# If we have too many 1's
if current_ratio > target_ratio:
    # Number of 1's to flip
    excess = int((current_ratio - target_ratio) * len(merged_df))
    idx_to_flip = merged_df[merged_df['Depression'] == 1].sample(excess, random_state=42).index
    merged_df.loc[idx_to_flip, 'Depression'] = 0


In [41]:
merged_df['Depression'].value_counts(normalize=True)

Depression
1.0    0.566169
0.0    0.433831
Name: proportion, dtype: float64

# **İçindekiler**

## 1. Proje Tanımı & Hedefler
## 2. Veri Kaynakları ve Birleştirme Stratejisi
## 3. Yüklemeler ve İlk Kontroller
## 4. Keşifsel Veri Analizi (EDA)
## 5. Veri Ön İşleme
## 6. Özellik Mühendisliği
## 7. Model Seçimi ve Eğitim (en az 3 model)
## 8. Model Değerlendirme ve Karşılaştırma
## 9. Model Optimizasyonu (Hyperparametre Tuning)
## 10. Açıklanabilirlik (Feature importance, SHAP)
## 11. Sonuçlar ve Raporlama / Sunum İçeriği
## 12. Ek Notlar: Etik, Veri Gizliliği, Kaynakça